## Demonstration of Mowing Detection ##

### IMPORT LIBRARIES ###

In [12]:
import json
from tqdm.notebook import tqdm

# Custom Libraries
import parcel_data_sources as pds
import time_series_sources as tss
import pre_processors as pps
import markers_processing as mp
import data_displayer as dd
import data_sink as ds
import marker_aggregation as ma
import scenario_evidence as se


# Utilities for computing geometric properties of the parcel
import geom_utils as gu

# Some widgets
from ipyfilechooser import FileChooser


### Open the option file ###

In [2]:
# get the option file using the FileChooser
fc = FileChooser('./')
display(fc)

FileChooser(path='.', filename='', show_hidden='False')

In [3]:
optionFilePath = fc.selected
optionFile = open(optionFilePath)
options = json.load(optionFile)

### Create the parcel data factory and the parcel data ###
*Parcel data factory: object responsible for creating the parcel data sources* 

*Parcel data sources: objects responsible for providing the list of parcels and related data*

In [4]:
parcel_df = pds.parcel_data_factory()
parcel_ds = parcel_df.get_parcel_data_source(options)
print("Parcel data source created \n")

# Retrieve the list of parcels (FIDs)
fid_list = parcel_ds.get_fid_list()

Parcel data source created 



### Create the time series source factory and the time series sources ###
*Time series source factory: object responsible for creating time series sources*

*Time series sources: objects responsible for providing the time series associated to the parcels* 

In [5]:
ts_source_factory = tss.time_serie_source_factory()
ts_sources = ts_source_factory.get_time_series_sources(options)
print("Time series sources created")

Time series sources created


### Build the pre-processing stage ###
*Pre-processor factory: object responsible for allocating the pre-processors*

*Pre-processors: objects responsible for pre-processing the input time series*

In [6]:
print(ts_sources[0].get_components())

['Field_ID', 'Date', 'ndvi_mean', 'ndvi_count', 'ndvi_std']


In [7]:
pp_factory = pps.processor_factory()
pre_processors = pp_factory.get_pre_processors(options)

### Build the marker detectors ###
*Marker detector factory: allocate the marker detectors accoring to the specifications of the option file*

*Marker detectors: determine specific features (markers) on the time series*

In [8]:
md_factory = mp.marker_detector_factory()
marker_detectors = md_factory.get_marker_detectors(options)

### Build the marker aggregator ###
*Object responsible for aggregating markers from different time series*

In [9]:
marker_agg = ma.marker_aggregator(options)

### Build the marker data sinks ###
*Write marker information to file (csv)*

In [10]:
if "marker-sink" in options :
    ms_option_list = options["marker-sink"]
    
    mk_sink = ds.marker_sink(ms_option_list[0])
    
    if len(ms_option_list) > 1 :
        agmk_sink = ds.marker_sink(ms_option_list[1])
    else:
        agmk_sink = None
else :
    mk_sink = None
    agmk_sink = None

### Build the data displayer ###
*Create summary plots and save them as png figures*

In [13]:
data_disp_factory = dd.data_displayer_factory()
data_disps = data_disp_factory.get_data_displayers(options)

### Scenario Evidence Writer ###

In [14]:
if "scenario-evidence" in options :
    scenario_ev = se.scenario_evidence(options["scenario-evidence"])
else :
    scenario_ev = None

---
---
## Main Processing Loop ##

In [15]:
for fid in tqdm(fid_list) :
    
    # get the parcel geometry
    parcel_geometry = parcel_ds.get_parcel( fid )
    
    # get the data related to the specific parcel
    parcel_data = {}
    
    for ts_source in ts_sources :
        data = ts_source.get_ts(fid)
        if len(data) > 0 :
            parcel_data[ts_source.get_signal_type()] = data
    
    # parcel_data is a dictionary with different pandas dataframes. Each
    # dataframe is indexed by a signal/TS key

    # Get the properties
    geom_prop = gu.get_geometric_properties( parcel_geometry )
    
    # eventually skip the parcel
    # ADD HERE LOGIC TO EVENTUALLY SKIP THE PARCEL
    # A similar approach can be adopted to check signal properties
    
    # Pre-process the data
    for pre_pro in pre_processors :
        parcel_data = {**parcel_data, **(pre_pro.process(parcel_data))}

    # Extract the markers
    markers = {}
    for marker_det in marker_detectors :
        markers = {**markers, **(marker_det.get_markers(parcel_data))}
      
    # Output the marker information
    if mk_sink is not None :
        # Output the marker information
        mk_sink.dump_marker_info(fid, markers)  

    # Create the summary graphs and save it to file
    for data_disp in data_disps :
        data_disp.dump_to_file(parcel_data, markers, fid)    

    # Aggregate the markers
    aggregated_markers = marker_agg.aggregate_markers(markers)
    # print([x.type for x in aggregated_markers])
    
    if agmk_sink is not None :
        agmk_sink.dump_marker_info(fid, aggregated_markers)

    # Write information at the parcel level
    if scenario_ev is not None :
        scenario_ev.dump(fid, aggregated_markers, geom_prop)

### Do some final clean-up ###

In [ ]:
if mk_sink is not None :    
    mk_sink.out_file.close()
    
if agmk_sink is not None :
    agmk_sink.out_file.close()
    
if scenario_ev is not None :
    scenario_ev.out_file.close()